# Data Extraction

Original Michelin restaurant data was extracted from Kaggle (https://www.kaggle.com/jackywang529/michelin-restaurants).

The data includes one, two, three Michelin star restaurants from the following regions:

Austria, California, Chicago, Croatia, Czech Republic, Denmark, Finland, Greece, Hong Kong, Hungary, Iceland, Macau, Norway, New York City, Poland, Ireland, Rio de Janeiro, Sao Paulo, South Korea, Singapore, Sweden, Taipei, Thailand, Washington DC, and United Kingdom.

The following regions are not included in the dataset:

Belgium, France, Germany, Italy, Japan, Luxembourg, Netherlands, Portugal, China, Spain, and Switzerland.

#### Import Dependencies

In [2]:
# Dependencies
import pandas as pd
from sqlalchemy import create_engine
import requests
import json

# Google developer API key
from config import g_key

#### Load CSV Files into Pandas Dataframes

In [3]:
# Create file paths
one_star_csv = "Resources/data/one-star-michelin-restaurants.csv"
two_star_csv = "Resources/data/two-stars-michelin-restaurants.csv"
three_star_csv = "Resources/data/three-stars-michelin-restaurants.csv"

In [4]:
# Load one star restaurants
raw_one_star_df = pd.read_csv(one_star_csv)
raw_one_star_df.head()

,name,year,latitude,longitude,city,region,zipCode,cuisine,price,url
0,Kilian Stuba,2019,47.348580,10.17114,Kleinwalsertal,Austria,87568,Creative,$$$$$,https://guide.michelin.com/at/en/vorarlberg/kl...
1,Pfefferschiff,2019,47.837870,13.07917,Hallwang,Austria,5300,Classic cuisine,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...
2,Esszimmer,2019,47.806850,13.03409,Salzburg,Austria,5020,Creative,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...
3,Carpe Diem,2019,47.800010,13.04006,Salzburg,Austria,5020,Market cuisine,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...
4,Edvard,2019,48.216503,16.36852,Wien,Austria,1010,Modern cuisine,$$$$,https://guide.michelin.com/at/en/vienna/wien/r...


In [5]:
# Load two star restaurants
raw_two_star_df = pd.read_csv(two_star_csv)
raw_two_star_df.head()

,name,year,latitude,longitude,city,region,zipCode,cuisine,price,url
0,SENNS.Restaurant,2019,47.83636,13.06389,Salzburg,Austria,5020,Creative,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...
1,Ikarus,2019,47.79536,13.00695,Salzburg,Austria,5020,Creative,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...
2,Mraz & Sohn,2019,48.23129,16.37637,Wien,Austria,1200,Creative,$$$$$,https://guide.michelin.com/at/en/vienna/wien/r...
3,Konstantin Filippou,2019,48.21056,16.37996,Wien,Austria,1010,Modern cuisine,$$$$$,https://guide.michelin.com/at/en/vienna/wien/r...
4,Silvio Nickol Gourmet Restaurant,2019,48.20558,16.37693,Wien,Austria,1010,Modern cuisine,$$$$$,https://guide.michelin.com/at/en/vienna/wien/r...


In [6]:
# Load three star restaurants
raw_three_star_df = pd.read_csv(three_star_csv)
raw_three_star_df.head()

,name,year,latitude,longitude,city,region,zipCode,cuisine,price,url
0,Amador,2019,48.25406,16.35915,Wien,Austria,1190,Creative,$$$$$,https://guide.michelin.com/at/en/vienna/wien/r...
1,Manresa,2019,37.22761,-121.98071,South San Francisco,California,95030,Contemporary,$$$$,https://guide.michelin.com/us/en/california/so...
2,Benu,2019,37.78521,-122.39876,San Francisco,California,94105,Asian,$$$$,https://guide.michelin.com/us/en/california/sa...
3,Quince,2019,37.79762,-122.40337,San Francisco,California,94133,Contemporary,$$$$,https://guide.michelin.com/us/en/california/sa...
4,Atelier Crenn,2019,37.79835,-122.43586,San Francisco,California,94123,Contemporary,$$$$,https://guide.michelin.com/us/en/california/sa...


In [7]:
# Row counts
print("In the raw data there are:")
print(f"{raw_one_star_df.shape[0]} rows for one star restaurants.")
print(f"{raw_two_star_df.shape[0]} rows for two star restaurants.")
print(f"{raw_three_star_df.shape[0]} rows for three star restaurants.")

In the raw data there are:
549 rows for one star restaurants.
110 rows for two star restaurants.
36 rows for three star restaurants.


#### Google API Calls

Our goal is to use the latitude and longitude provided by the original csv data to make API calls to the Google Places API. We will locate the nearest lodging to each Michelin restaurant and extract the name of these hotels to add to the dataset.

In [8]:
# Subset original data frames
one_star_hotel_df = raw_one_star_df[['name', 'latitude', 'longitude']].copy()
two_star_hotel_df = raw_two_star_df[['name', 'latitude', 'longitude']].copy()
three_star_hotel_df = raw_three_star_df[['name', 'latitude', 'longitude']].copy()

In [9]:
# Display copies
one_star_hotel_df.head()

,name,latitude,longitude
0,Kilian Stuba,47.348580,10.17114
1,Pfefferschiff,47.837870,13.07917
2,Esszimmer,47.806850,13.03409
3,Carpe Diem,47.800010,13.04006
4,Edvard,48.216503,16.36852


In [10]:
# Set up url for Google Places API Calls
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location" : "",
    "rankby" : "distance",
    "type": "lodging",
    "key": g_key,
}

In [11]:
# Run a test API call to check JSON format
lat = 48.25406
lng = 16.35915
params['location'] = f"{lat}, {lng}"
response = requests.get(base_url, params=params).json()
print(json.dumps(response, indent = 4, sort_keys = True))

{
    "html_attributions": [],
    "next_page_token": "ATtYBwKD_2o9AfozyOcnERAWwR_3POoVYve9w8n998bFsF2m5QNO4xDsUCPgKzKlNURvQTHRVJHcGdKPY4OVuxtKdTfUq2DrMiP9_xJA5H6g8_SyD0p5OMn0qAQuKFqoLp2KhtsD05iyNRjaeDt6Sk1J8jhbWqLRZuurYPAejRQ8abFc8H5yFqEjPik35qraepx_hIQlz1iqePZJgsKoVUZJo1gfWHkvaedrpHbRGswuDJoqFv69nVpKdFB7sFl1eoxUQ8gYMgDYXtC_zX5Tc-K0CGNNlktLa1PFbswTq0XjTJ0FrMY9QRj2Qkirlh8qWjaeRQqT5R9jY6cPIXJCr_ajE4D0zzSiIYIJkDu-ajwdyw307qkNYc4FLxDLPuhXz0Eojkr5Ab_66cEOeiPP7lUA6qE7fqZikF8vG0B7dvnBjiqxCueb2lfCsHO3Lw",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 48.25690939999999,
                    "lng": 16.3589786
                },
                "viewport": {
                    "northeast": {
                        "lat": 48.25825088029149,
                        "lng": 16.3602773802915
                    },
                    "southwest": {
                        "lat": 48.255552919

In [14]:
# Loop through rows for coordinates to make API call for one star
for index, row in one_star_hotel_df.iterrows():
    lat = row.latitude
    lng = row.longitude
    params['location'] = f"{lat}, {lng}"
    response = requests.get(base_url, params=params).json()
    one_star_hotel_df.loc[index, "Hotel_Name"] = response["results"][0]["name"]
    print(f"Hotel Added")

Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hote

In [15]:
one_star_hotel_df.head()

,name,latitude,longitude,Hotel_Name
0,Kilian Stuba,47.348580,10.17114,Ferienwohnungen Braun
1,Pfefferschiff,47.837870,13.07917,Ferienhaus Essl-Redecsy
2,Esszimmer,47.806850,13.03409,Exerzitienhaus
3,Carpe Diem,47.800010,13.04006,arthotel blaue gans salzburg
4,Edvard,48.216503,16.36852,Palais Hansen Kempinski Vienna


In [17]:
# Loop through rows for coordinates to make API call for two star
for index, row in two_star_hotel_df.iterrows():
    lat = row.latitude
    lng = row.longitude
    params['location'] = f"{lat}, {lng}"
    response = requests.get(base_url, params=params).json()
    two_star_hotel_df.loc[index, "Hotel_Name"] = response["results"][0]["name"]
    print(f"Hotel Added")

Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hotel Added
Hote

In [18]:
two_star_hotel_df.head()

,name,latitude,longitude,Hotel_Name
0,SENNS.Restaurant,47.83636,13.06389,Inovum
1,Ikarus,47.79536,13.00695,Kärntner Chalets
2,Mraz & Sohn,48.23129,16.37637,Wohnen im Kapellenhof
3,Konstantin Filippou,48.21056,16.37996,PuzzleHotel Apartments Postgasse
4,Silvio Nickol Gourmet Restaurant,48.20558,16.37693,Palais Coburg


In [12]:
# Loop through rows for coordinates to make API call for three star
for index, row in three_star_hotel_df.iterrows():
    lat = row.latitude
    lng = row.longitude
    params['location'] = f"{lat}, {lng}"
    response = requests.get(base_url, params=params).json()
    three_star_hotel_df.loc[index, "Hotel_Name"] = response["results"][0]["name"]
     print(f"Hotel Added")

In [13]:
three_star_hotel_df.head()

,name,latitude,longitude,Hotel_Name
0,Amador,48.25406,16.35915,Zum Alten Stadttor
1,Manresa,37.22761,-121.98071,Inspired Leadership Group
2,Benu,37.78521,-122.39876,W San Francisco
3,Quince,37.79762,-122.40337,SnapTravel
4,Atelier Crenn,37.79835,-122.43586,The gula place


In [19]:
one_star_hotel_df.to_csv("Resources/data/one_star_hotel.csv")
two_star_hotel_df.to_csv("Resources/data/two_star_hotel.csv")
three_star_hotel_df.to_csv("Resources/data/three_star_hotel.csv")

#### Web Scraping

In [ ]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import time

In [ ]:
def michelin_web_scrape(specials_df):
    
    # Set up splinter if this is the first call of function
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)
    
    # Loop through 
    for index, row in specials_df.iterrows():

        # Visit url and scrape specialties data
        url = row.url
        browser.visit(url)
        time.sleep(1)
        html = browser.html
        soup = bs(html, 'html.parser')
        try:
            scrape_data = soup.find('ul', class_ ="restaurant-details__text-componets--list")
            specialties = scrape_data.find_all('li')


            # Loop through scrape results and append text results to a specials list
            foods = []
            for special in specialties:
                foods.append(special.text)
            print(foods)
            # Add specials list for restaurant to special dataframe
            specials_df.at[index, 'specialties'] = foods
            print('Above specials added successfully.')
        except:
            print("Information not found")
            specials_df.at[index, 'specialties'] = None
            

    # Close browser
    browser.quit()
    print("Scraping complete.")
       
        

##### One Star Scrape

In [ ]:
# Create new dataframe with restaurant name and url info to hold scraped data
one_star_specialties_df = raw_one_star_df[['name','url']].copy()
one_star_specialties_df["specialties"] = ""
one_star_specialties_df.head()

In [ ]:
# Use the scrape function we created to get specialties for each restaurant if applicable
# michelin_web_scrape(one_star_specialties_df)

In [ ]:
one_star_specialties_df.head()

##### Two Star Scape

In [ ]:
# Create new dataframe with restaurant name and url info to hold scraped data
two_star_specialties_df = raw_two_star_df[['name','url']].copy()
two_star_specialties_df["specialties"] = ""
two_star_specialties_df.head()

In [ ]:
# Use the scrape function we created to get specialties for each restaurant if applicable
michelin_web_scrape(two_star_specialties_df)

In [ ]:
two_star_specialties_df.head()

##### Three Star Scrape

In [ ]:
# Create new dataframe with restaurant name and url info to hold scraped data
three_star_specialties_df = raw_three_star_df[['name','url']].copy()
three_star_specialties_df["specialties"] = ""
three_star_specialties_df.head()

In [ ]:
# Use the scrape function we created to get specialties for each restaurant if applicable
michelin_web_scrape(three_star_specialties_df)

In [ ]:
three_star_specialties_df.head(36)

# Data Transformation

# Data Loading